# Анализ эксперимента

### Цель: оценить результат эксперимента (сравнить средние значения двух выборок) с помощью доверительного интервала.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from scipy import stats as st
import numpy as np
from numpy.random import binomial
from numpy.random import normal

## Сгенерируем семплы

Два набора данных, распределенных по биномиальному закону. В каждом эксперименте по 100 наблюдений.

In [ ]:
P1 = 0.1
P2 = 0.11
N1 = 100
N2 = 100

In [ ]:
sample1 = binomial(1, P1, N1)
sample2 = binomial(1, P2, N2)

In [ ]:
mean1 = np.mean(sample1)
mean2 = np.mean(sample2)
diff = mean1 - mean2
print('mean1: %s' % mean1)
print('mean2: %s' % mean2)
print('mean1 - mean2: %s' % diff)

**Истинные вероятности отличаются на 0.01. Но в эксперименте получилось, что разность другая. Поэтому только на разность средних опираться нельзя, необходимо смотреть на распределения.**

## Оценим выборочную дисперсию

Предполагаем, что выборки из одного и того же распределения. Считаем дисперсию для этого распределения.

In [ ]:
std_pool = np.sqrt(mean1 * (1 - mean1) / N1 + mean2 * (1 - mean2) / N2)

In [ ]:
confidence = 0.975
z = st.norm.ppf(confidence)

margin = z * std_pool
print('z: %s' % z)

## Случайно ли различие в средних?

Гипотеза H0 состоит в том, что распределения одинаковы. Отвергаем её, если |diff| > margin.

In [ ]:
print('margin: %s' % margin)
print('diff: %s' % diff)
print('Are they different (diff > margin)? -> %s' % (abs(diff) > margin))

Эквивалентная, но по-другому записанная проверка

In [ ]:
z_stat = abs(diff) / std_pool
print('Эквивалентная проверка:', z_stat > z)

***Результат: статистика говорит, что выборки из одного распределения.***

То есть если в каждом эксперименте 100 наблюдений, этого недостаточно для того, чтобы статистически выявить различия в 1% вероятности.

### Задание.

Попробуйте взять N1=N2=1000 наблюдений и посмотреть на результат проверки гипотезы? Запустите код несколько раз. Сколько раз получаем верный результат?

Теперь возьмите N1=N2=10000.

Затем возьмите N1=N2=100000.

### Определить минимальное количество наблюдений в эксперименте, при котором различие вероятностей в 1% будет значимым: требуется построить критерий мощности 90%.

Хотим найти минимальное число наблюдений в эксперименте, чтобы из 500 экспериментов хотя бы в 90% случаев мы приняли верное решение, то есть отклонили гипотезу о том, что выборки из одного распределения.

In [ ]:
#TODO: сгенерировать два набора экспериментов, в каждом по num_exp экспериментов с N наблюдениями
#внутри эксперимента и с вероятностями P1 и P2.
#Для каждого размера выборки N посчитать долю выборок, для которых abs(diff) > margin.
#Нам нужно минимальное N такое, что эта доля >= 0.9 (N ищем в цикле).

from tqdm import tqdm_notebook

num_exp = 500
Res = []

for N in tqdm_notebook(np.arange(10000,100000,1000)): #ищем с маленьким шагом
    experimentsA = binomial(1, P1, (num_exp, N))
    experimentsB = binomial(1, P2, (num_exp, N))
    
    meanA = np.array([np.mean(exp) for exp in experimentsA])
    meanB = np.array([np.mean(exp) for exp in experimentsB])
    
    diff = abs(meanA - meanB)
    std_pool = np.sqrt(meanA * (1 - meanA) / N + meanB * (1 - meanB) / N)
    
    z_stat = diff / std_pool
    
    res = sum(z_stat > z) / num_exp #доля верно отклоненных гипотез

    Res.append(res)
    
    if res >= 0.9:
        print(N)
        break

In [ ]:
Res

In [ ]:
plt.plot(Res)